In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint

In [132]:
personal_data = pd.read_csv('./data/personal_train.csv')
personal_data = personal_data.drop(['Unnamed: 0'], axis=1)
other_data = pd.read_csv('./data/other_train.csv')
other_data = other_data.drop(['Unnamed: 0'], axis=1)

merged = pd.merge(personal_data,other_data,on=["name","address"])
duplicated = merged.duplicated(subset=["name","address"],keep=False)
duplicates = pd.DataFrame(columns=merged.columns)
indexes_to_drop = []

for index, row in merged.iterrows():
    if(duplicated[index]):
        duplicates = duplicates.append(merged.iloc[index],ignore_index=True)
        indexes_to_drop.append(index)

duplicates.sort_values(["name","address"])

for index in indexes_to_drop:
    merged = merged.drop(merged.index[index])
    
lastDuplicate = duplicates.iloc[0]["name"]
columns = duplicates.columns.values

def getUniqueRow(startIndex,endIndex):
    row_dict = {}
    for index in range(startIndex,endIndex):
        row_num = randint(startIndex,endIndex-1)
        for column in duplicates.columns:
            if(duplicates.iloc[row_num])
    

count = 0
for index,row in duplicates.iterrows():
    entry = duplicates.iloc[index]
    name = entry["name"]
    if(name == lastDuplicate):
        count += 1
    else:
        getUniqueRow(index-count,index)
        lastDuplicate = name
        count = 1
        
        

name
address
age
sex
date_of_birth
query hyperthyroid
FTI measured
education
lithium
TT4
T4U
capital-loss
capital-gain
tumor
TSH
T3
fnlwgt
hours-per-week
relationship
sick
workclass
TT4 measured
class
marital-status
goitre
native-country
hypopituitary
medical_info
on antithyroid medication
referral source
education-num
occupation
TBG measured
TBG
race
FTI
name
address
age
sex
date_of_birth
query hyperthyroid
FTI measured
education
lithium
TT4
T4U
capital-loss
capital-gain
tumor
TSH
T3
fnlwgt
hours-per-week
relationship
sick
workclass
TT4 measured
class
marital-status
goitre
native-country
hypopituitary
medical_info
on antithyroid medication
referral source
education-num
occupation
TBG measured
TBG
race
FTI
name
address
age
sex
date_of_birth
query hyperthyroid
FTI measured
education
lithium
TT4
T4U
capital-loss
capital-gain
tumor
TSH
T3
fnlwgt
hours-per-week
relationship
sick
workclass
TT4 measured
class
marital-status
goitre
native-country
hypopituitary
medical_info
on antithyroid medi

relationship
sick
workclass
TT4 measured
class
marital-status
goitre
native-country
hypopituitary
medical_info
on antithyroid medication
referral source
education-num
occupation
TBG measured
TBG
race
FTI
name
address
age
sex
date_of_birth
query hyperthyroid
FTI measured
education
lithium
TT4
T4U
capital-loss
capital-gain
tumor
TSH
T3
fnlwgt
hours-per-week
relationship
sick
workclass
TT4 measured
class
marital-status
goitre
native-country
hypopituitary
medical_info
on antithyroid medication
referral source
education-num
occupation
TBG measured
TBG
race
FTI
name
address
age
sex
date_of_birth
query hyperthyroid
FTI measured
education
lithium
TT4
T4U
capital-loss
capital-gain
tumor
TSH
T3
fnlwgt
hours-per-week
relationship
sick
workclass
TT4 measured
class
marital-status
goitre
native-country
hypopituitary
medical_info
on antithyroid medication
referral source
education-num
occupation
TBG measured
TBG
race
FTI
name
address
age
sex
date_of_birth
query hyperthyroid
FTI measured
education
lit